?  =  variable name, possibly with indexing/slicing  
??? = function / class name  
?????? = complex expression containing variables, functions etc  
???B??? = complex expression that must contain "B"  

# What to do with 10,000 simultaneous neurons?  

# Section 1. Loading and preparing. 

### 1.0 Load in the Python libraries

In [ ]:
import os # os stands for "operating system" and includes read/write routines etc. 
import numpy as np # by far the most used library for everyday computation
from matplotlib import pyplot as plt # plotting functions. second most important library.
%matplotlib inline 

# scipy is an "expansion" of numpy with more specialized functions
from scipy import stats # here we import a sub-library of stats functions
from ???.??? import ??? # here we import gaussian_filter
f
# sklearn is the "machine learning" library. Has lots of good stuff. Try to find PCA 
from sklearn.??? import ???

In [ ]:
# let's inspect the function we imported
gaussian_filter

In [ ]:
# more info about this function
gaussian_filter?

In [ ]:
# anything else we imported you want to inspect here?

### 1.1 Load the metadata

In [ ]:
# this cell loads Suite2p configuration file "ops", and stimulus information (mov, iframe)
mname, datexp, blk = 'TX39', '2019_05_31', '1' # this assignment works! 

root      = '/home/neuraldata/data/meso'
mov       = np.load(os.path.join(root, 'mov.npy')) # mov contains the sparse noise frames
iframe    = np.load(os.path.join(root, 'iframe.npy')) # iframe[n] is the microscope frame for the image frame n

# ops is a "pickled" dictionary, not just an array
ops = np.load(os.path.join(root, 'suite2p', 'combined', 'ops.npy'), allow_pickle=True).item() 

In [ ]:
# dictionaries have "keys". Figure out how to list the keys. 
ops.???

In [ ]:
%matplotlib notebook # if you need to zoom into a figure, this is the "interactive" mode of matplotlib
# you cannot go back and forth twice between interactive and inline modes (restart nootebook instead)

# display the maximum projection image from ops. You need to access ops like a dictionary. 
plt.figure(figsize=(8,8))
plt.imshow(???) 

In [ ]:
# now let's adjust this plot. 
# Set the saturation of the image to 5000, the colormap to gray, and the aspect ratio to "auto"
plt.figure(figsize=(8,8))
plt.imshow(???, ???=5000, ??? = 'gray', ??? ='auto') 

### 1.2 Load the neural data

In [ ]:
spks = np.load(os.path.join(root, 'suite2p', 'combined', 'spks.npy'))
stat = np.load(os.path.join(root, 'suite2p', 'combined', 'stat.npy'), allow_pickle=True)

ypos = [stat[k]['med'][0] for k in range(len(stat))] # this is equivalent to a for loop, but written more succintly
xpos = [stat[k]['med'][1] for k in range(len(stat))]

# ypos, xpos are Python "lists". Before we can operate on them, they should be converted to Numpy "arrays".
ypos, xpos = np.???(ypos), np.???(xpos)

ypos, xpos = ypos/.5, xpos/.75 # this recording had 0.5 pixels/um in Y and 0.75 pixels/um in X

print('total neurons %d'%len(stat))
print('recorded from an area of %2.2f um by %2.2f um'%(np.ptp(ypos), np.ptp(xpos)))

In [ ]:
# make a "scatter plot" overlaid on top of the image in the cell above. 
# because we are in notebook mode, this will plot to the last interactive plot
plt.???(xpos*.75, ypos*.5) 

In [ ]:
# switch back to inline plots so we can make them bigger
%matplotlib inline

### 1.3 By the end of this section you should have spks, Timeline, xpos, ypos in your workspace. 
Try to explore the contents of these variables, for example:

In [ ]:
print(stat[0].keys())

In [ ]:
print(np.min(ypos), np.max(ypos), np.min(xpos), np.max(xpos))

In [ ]:
# use this empty cell to determine how many timepoints are in the dataset, and how many images have been presented 
print(?.shape)

# Section 2. Plotting and visualizing the data

In [ ]:
# POSITIONS OF ALL NEURONS
plt.scatter(xpos, -ypos, s = 1) # note we have to invert the Ypos
plt.xlabel('X position (um)', fontsize=20) # this is how you make axis labels
plt.ylabel('Y position (um)', fontsize=20) 
plt.axis('square') # make both axes the same size
plt.show() # when you are done

In [ ]:
# display movie frame 100
plt.imshow(?, cmap='gray')
plt.title('example frame') # without plt show, the result of the last command is displayed

In [ ]:
dt     = 1 # time offset between stimulus and neural activity (don't change)
ivalid = iframe+dt<spks.shape[-1] # remove timepoints that fall after the neural recording ended
iframe = iframe[ivalid] 
mov = mov[:, :, ivalid]

# subsample the neural timepoints corresponding to these movie frames
S   = spks[:, ?+?]

# z-score the neural activity before doing anything. axis specifies the dimension. 
S = stats.zscore(S, axis=1) 

In [ ]:
plt.figure(figsize=(16,6))
# display the first 100 neurons and the first 500 timepoints from S
plt.imshow(?, vmax = 3, vmin = -3, aspect='auto', cmap = 'gray')
plt.title('sample of the neural data matrix')
plt.ylabel('neurons') 
plt.xlabel('time points')

# Section 3. Receptive fields  (ON - OFF)

### 3.1 First, compute ordinary ON - OFF receptive fields

In [ ]:
nn     = 8867
NN, NT = S.shape 
S0     = S[nn, :]

X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints. -1 means the dimension is automatically inferred. 
X = X-0.5 # subtract the gray level
X = ???.???(?, axis=1) # z-score X along axis 1
X = X / NT**.5  # normalize Xto unit norm. 
npix = X.shape[0]

B0 = X @ S0 # stimulus triggered receptive field for one neuron
# reshape B0 into the vertical and horizontal dimensions of mov
B0 = np.reshape(B0, [?, ?])

In [ ]:
plt.figure(figsize=(16,3))
plt.subplot(1,2,1)
# plot the receptive field for each neuron. Use a "blue, white, red" colormap
plt.imshow(B0, aspect='auto', vmin=-12, vmax=12, cmap = '???') 
plt.title('raw')

plt.subplot(1,2,2)
B1 = ???(B0, [.5, .5]) # smooth each receptive field a little
plt.imshow(B1, aspect='auto', vmin=-12, vmax=12, cmap = '???') # plot the receptive field for each neuron
plt.title('smoothed')
plt.show()

In [ ]:
# go back above and take the absolute value of X after subtracting the gray level
 X = np.?(X) # take the absolute value of X

### 3.2 Receptive fields for all neurons

In [ ]:
NN, NT = S.shape 

# compute X as before
X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints
X = X-0.5 # subtract the gray level
X = stats.zscore(X, axis=1)/NT**.5  # z-score each pixel separately
npix = X.shape[0]


B0 = X @ S.T # get the receptive fields for each neuron        
B0 = np.reshape(B0, [mov.shape[0], mov.shape[1], NN])
# smooth each receptive field by 0.5 gaussian standard deviation along horizontal and vertical dimension
# smooth by 0 gaussian standard deviation along the neurons dimension. 
B0 = gaussian_filter(B0, [?, ?, ?]) 

B00 = B0 # make a copy of B0 for later

In [ ]:
# create a figure of vertiical size 18 and horizontal size 10.
???.???(???=(?, ?))
np.random.seed(1) # set the seed for the random number generator, so that our random samples are reproducible
rperm = np.random.permutation(NN) # choose a randomly permuted set of neurons

isort = rperm
#isort = np.argsort(np.min(B0, axis=(0,1))) # try this one next! 
#isort = np.argsort(np.max(B0, axis=(0,1)))[::-1] # this is how we get a "descending" sort

# now sort by the (max - min) range of each receptive field
#isort = np.argsort(??????)[::-1]

for j in range(8*7):
    plt.subplot(8,7,j+1)
    # show the receptive field for unit isort[j]
    plt.imshow(?, aspect='auto', vmin=-6, vmax=6, cmap = 'bwr') # plot the receptive field for each neuron
    plt.axis('off')
    
plt.show()

# Section 4. Receptive fields (ON + OFF)

### 4.1 Do the neurons care about contrast SIGN ?

In [ ]:
# like before, but with the absolute value
X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints
X = X-0.5 # does not matter if a pixel is black (0) or white (1)
X = np.abs(X) # response is the same for ON and OFF squares
X = stats.zscore(X, axis=1)/NT**.5  # z-score each pixel separately

B0 = X @ S.T # get the receptive fields for each neuron

B0 = np.reshape(B0, [mov.shape[0], mov.shape[1], -1])
B0 = gaussian_filter(B0, [.5, .5, 0]) # smooth each receptive field a little

In [ ]:
plt.figure(figsize=(18, 10))
# how do we make sure we get the same random sample of neurons as before?
???.???.???(?)
rperm = np.random.permutation(NN) # choose a randomly permuted set of neurons

isort = rperm
#isort = np.argsort(np.min(B0, axis=(0,1)))
#isort = np.argsort(np.max(B0, axis=(0,1)))[::-1]
#isort = np.argsort(???)[::-1]

for j in range(7*8):
    plt.subplot(8,7,j+1)
    plt.imshow(?, aspect='auto', vmin=-6, vmax=6, cmap = 'bwr') # plot the receptive field for each neuron
    plt.axis('off')
    
plt.show()

### 4.2 Let's average the ON+OFF receptive fields over groups of neurons

In [ ]:
sig = 50 # average neurons within this distance
B = np.zeros(B0.shape)
for j in range(NN):
    # for each neuron, let's find all others within sig distance in um
    ds = (ypos[j] - ?)**2 + (xpos[j] - ?)**2
    ix = ds**.5 < sig
    B[:,:,j] = np.mean(?, axis=-1) # average receptive fields of ix=True neurons
    
Amax = ???B??? # compute the maximum response of each neuron

In [ ]:
plt.figure(figsize=(18, 10))
np.random.seed(1) # set the seed for the random number generator, so that our random samples are reproducible
rperm = np.random.permutation(NN) # choose a random set of neurons

for j in range(7*7):
    plt.subplot(7,7,j+1)
    plt.imshow(B[:,:,rperm[j]], aspect='auto', vmin=-3, vmax=3, cmap = 'bwr') # plot the receptive field for each neuron
    plt.axis('off')
    
plt.show()

### 4.3 Finally, let's display the retinotopy over the recording area

In [ ]:
imax = np.argmax(np.reshape(B, [-1,NN]), axis=0) # find the pixel corresponding to the max response for each cell
ly, lx, nstim = ?.??? # get the dimensions of the stimulus

# use "unravel_index" to go from a linear index imax to a two-dimensional index (ymax,xmax)
ymax, xmax = ???imax??? # unravel an index for a (1,ly*lx) array, to two indices for an (ly, lx) array
xmax = np.???(17, ?) # threshold xmax at 17 (middle of the screen)

plt.figure(figsize=(12,6))

plt.subplot(1,2,1, facecolor= ?) # figure background should be black
plt.scatter(xpos, -ypos, s = ?, c = ?, cmap='viridis') 
# dot size should be proportional to max response Amax times a factor
# the dot color should be determined by the preferred horizontal position
plt.title('Horizontal')

plt.subplot(1,2,2, facecolor=?) # figure background should be black
plt.scatter(xpos, -ypos, s = ?, c = ?, cmap='viridis') 
# same as above for preferred vertical position
plt.title('Vertical')

plt.tight_layout()

plt.show()